
 # *Data Mining e Machine Learning II*


*Instituição:* Centro Universitário IESB

*Curso:* Pós Graduação Ciência de Dados - Campus BSB/Asa Sul

*Disciplina:* Data Mining e Machine Learning II

*Orientador:* Marcos Vinicius Guimarães

*Aluna:* Adriana Maria Santos Viana


****INTRODUÇÃO****



No presente estudo, utilizamos a base HMEQ_Data, encontrada na plataforma Kaggle. 
A base traz dados sobre o departamento de crédito ao consumidor de um banco e foram usadas para auxiliar a decisão da aprovação das linhas de crédito a partir de dados estatisticos.

A variável BAD foi utilizada como Target(alvo) e traz as informações divididas em "Dummies" com os valores 1 = cliente inadimplente no empréstimo e 0 = empréstimo reembolsado.

Realizado análise exploratória dos dados e aplicado modelo de predição e tecnicas para melhorias dos parametros de estimativa. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
#Importando as bibliotecas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from  sklearn.model_selection import train_test_split  

from sklearn.metrics import mean_squared_log_error

from sklearn.ensemble import RandomForestClassifier

import scikitplot as skplt 

from sklearn.model_selection import train_test_split

In [ ]:
#Importando a base de dados

base = pd.read_csv('/kaggle/input/hmeq-data/hmeq.csv')

base.head(10)

In [ ]:
base.info()

In [ ]:
#Transformando REASON em Dummy
pd.get_dummies(base['REASON']).iloc[:4]

In [ ]:
#Transformando JOB em Dummy
pd.get_dummies(base['JOB']).iloc[:5]

In [ ]:
base = pd.get_dummies(base, columns=['REASON', 'JOB'])

In [ ]:
base.head().T

In [ ]:
#Confirmar a transformação da variável BAD
base.info()

In [ ]:
#Análise estistica dos dados.
base[base['BAD']==0].drop('BAD', axis=1).describe().style.format("{:.2f}")

In [ ]:
#Correlação entre as variáveis
base.corr()

In [ ]:
#Analise da variavel BAD
sns.countplot(base['BAD'])

No gráfico é possivel identifica que 0 = empréstimo reembolsado é maior que os valores para 1 = cliente inadimplente no empréstimo.

In [ ]:
# Histograma com a distribuição das variáveis
base.hist()
plt.show()

In [ ]:

base.iloc[:,[0]]

In [ ]:
#Retirar os valores nulos

base.dropna(inplace=True)

In [ ]:
#Confirmando a retirada dos valores NaN
base.isnull().sum()

In [ ]:
#Grafico das variaveis LOAN e BAD com a quantidade de emprestimos e os clientes inadimplente por empréstimo e 
#emprestimos reembolsados, onde 0 = Cliente inadimplente por emprestimos | 1 = Empréstimos reembolsados.

plt.suptitle("Quantidade de emprestimos e tipos de emprestimos")
sns.stripplot(x='BAD', y='LOAN', data=base)

In [ ]:
# Definindo as features 
feats = [c for c in base.columns if c not in ['LOAN', 'BAD']]
feats

In [ ]:
#Separando o dataframe 

#Importando o train_test_split
from sklearn.model_selection import train_test_split

#Primeiro treino e teste
train, test = train_test_split(base, test_size=0.20, random_state=42)

#Treino e validação
train, valid = train_test_split(train, test_size=0.20, random_state=42)

train.shape, valid.shape, test.shape

In [ ]:
#Importando o RandomForest
from sklearn.ensemble import RandomForestClassifier

#Importando o modelo
rf = RandomForestClassifier(n_estimators=200, random_state=42)

In [ ]:
# Treinando o modelo

rf.fit(train[feats], train['BAD'])

In [ ]:
#Primeiro treino e teste
train, test = train_test_split(base, test_size=0.20, random_state=42)

In [ ]:
#Analisando o desempenho do modelo

#Importando metricas

from sklearn.metrics import accuracy_score

In [ ]:
#Avaliando os dados de validacao

#Obter as previsões da base de validação
pred = rf.predict(valid[feats])

#Verificar acurácia
accuracy_score(valid['BAD'], pred)

In [ ]:
#Avaliando os dados de teste

# obter as previsões dos dados de teste
pred_test = rf.predict(test[feats])

#Verificar a acurácia
accuracy_score(test['BAD'], pred_test)

In [ ]:
# Olhar o dataFrame  completo
base['BAD'].value_counts(normalize=True)

In [ ]:
base['BAD'].astype('category').cat.categories

In [ ]:
# acessando os mapeamentos das categorias
base['BAD'].astype('category').cat.codes

In [ ]:
plt.figure(figsize=(10, 9))

#Primeiro modelo criado
pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh()

In [ ]:
# matriz de confusão
#importar biblioteca de matriz de confusão

import scikitplot as skplt 

In [ ]:
# Dados de validação
#comparar onde acertou ou não. Falsos positivos e falsos negativos

skplt.metrics.plot_confusion_matrix(valid['BAD'], pred)

Após gerar a matriz de confusão é possivel identificar que o modelo classificou 511 instancias que foram previstas corretamente. Possivelmente devido a retirada dos valores missing.

In [ ]:
#Dados de teste | Matriz de confusão
skplt.metrics.plot_confusion_matrix(test['BAD'], pred_test)

Após gerar a matriz de confusão é possivel identificar que o modelo classificou 638 instancias que foram previstas corretamente. 
Possivelmente devido a retirada dos valores missing.

In [ ]:
# XGBoost

# Importar o modelo
from xgboost import XGBClassifier

# Instanciar o modelo
xgb = XGBClassifier(n_estimators=200, n_jobs=-1, random_state=42, learning_rate=0.05)

In [ ]:
# Usando o cross validation
scores = cross_val_score(xgb, train[feats], train['BAD'], n_jobs=-1, cv=5)

scores, scores.mean()

In [ ]:
# Usando o XGB para treinamento e predição
xgb.fit(train[feats], train['BAD'])

In [ ]:
# Fazendo predições
preds = xgb.predict(test[feats])

In [ ]:
# Medir o desempenho do modelo
from sklearn.metrics import accuracy_score

accuracy_score(test['BAD'], preds)

Após execução dos dois modelos, o melhor modelo foi o XGBoost pois apresentou 0.94.
